### Operationalizing ML Models
John Hoff  
Machine Learning Architect  
jhoff@productiveedge.com
# Step 4: Use the Model
![Step 1: Prepare](https://drive.google.com/uc?export=view&id=1lg6Wymz3XDy9Pe_v4-H1MRAj_RMUF0mI)

This step will call the deployed models using the skewed dataset. It can call a local Spark model or a model deployed to ACI or AKS.

_Please Note: Do not "Run All". There are two different options for model image deployment._

In [2]:
import azureml
from azureml.core import Workspace
from azureml.core.webservice import Webservice
import json
import mlflow.azureml
import mlflow.spark
import numpy as np
import requests

In [3]:
workspace_name = '----YOUR WORKSPACE NAME----'
workspace_location = '----YOUR WORKSPACE LOCATION----'
resource_group = '----YOUR RESOURCE GROUP----'
subscription_id = '----YOUR SUBSCRIPTION ID----'

mlflow_run_id = '----YOUR MLFLOW RUN ID----'

aci_webservice_name = 'acisparkmodel'
aks_cluster_name = 'akssparkcluster'
aks_webservice_name = 'akssparkmodel'

chunks = 250

## Option 1: Use the Local Model
This will load the saved Spark MLlib model locally on the cluster.  Predictions can then be made with this model in a manner similar to how the model was scored after being trained.

In [5]:
# Load the skewed data to send to the model
input_data = spark.sql("select * from bank_marketing_skewed order by rand()")

# Load the model using the run id
model = mlflow.spark.load_model('runs:/' + mlflow_run_id + '/spark_model')

# The predictions of the model are made by transforming the input data
predictions = model.transform(input_data)

# Display the first 10 predictions
display(predictions.select('label', 'rawPrediction', 'probability', 'prediction').limit(10))

label,rawPrediction,probability,prediction
1.0,"List(1, 2, List(), List(4.950112517436611, 15.04988748256339))","List(1, 2, List(), List(0.24750562587183053, 0.7524943741281696))",1.0
0.0,"List(1, 2, List(), List(10.007720208828097, 9.992279791171905))","List(1, 2, List(), List(0.5003860104414048, 0.49961398955859526))",0.0
0.0,"List(1, 2, List(), List(19.357530835414316, 0.6424691645856856))","List(1, 2, List(), List(0.9678765417707158, 0.03212345822928428))",0.0
0.0,"List(1, 2, List(), List(19.29816400482516, 0.701835995174842))","List(1, 2, List(), List(0.9649082002412579, 0.03509179975874209))",0.0
0.0,"List(1, 2, List(), List(19.357530835414316, 0.6424691645856856))","List(1, 2, List(), List(0.9678765417707158, 0.03212345822928428))",0.0
0.0,"List(1, 2, List(), List(19.39407922585162, 0.6059207741483815))","List(1, 2, List(), List(0.969703961292581, 0.030296038707419076))",0.0
0.0,"List(1, 2, List(), List(19.037794129613395, 0.9622058703866044))","List(1, 2, List(), List(0.9518897064806697, 0.04811029351933022))",0.0
0.0,"List(1, 2, List(), List(19.037794129613395, 0.9622058703866044))","List(1, 2, List(), List(0.9518897064806697, 0.04811029351933022))",0.0
0.0,"List(1, 2, List(), List(19.391837850880684, 0.6081621491193189))","List(1, 2, List(), List(0.969591892544034, 0.03040810745596594))",0.0
1.0,"List(1, 2, List(), List(16.39104750457342, 3.6089524954265846))","List(1, 2, List(), List(0.8195523752286707, 0.18044762477132917))",0.0


## Option 2: User the ACI Model
Connect to the Azure ML workspace, find the ACI webservice running the spark model, and run the skewed input data through the model.

In [7]:
# Connect to the Azure ML Workspace
workspace = Workspace.create(name = workspace_name,
                             location = workspace_location,
                             resource_group = resource_group,
                             subscription_id = subscription_id,
                             exist_ok=True)

# Get the ACI Webservice containing our model
aci_webservice = None
for service in Webservice.list(workspace):
  if service.name == aci_webservice_name:
    aci_webservice = service

In [8]:
input_data = spark.sql("select * from bank_marketing_skewed order by rand()")

In [9]:
for chunk in np.array_split(input_data.toPandas(), chunks):
  print('.', end='')
  predictions = aci_webservice.run(chunk.to_json(orient='split'))

print('')
print('Scoring Complete!')

..........................................................................................................................................................................................................................................................
Scoring Complete!

## Option 3: User the AKS Model
Connect to the Azure ML workspace, find the AKS webservice running the spark model, and run the skewed input data through the model.

In [11]:
# Connect to the Azure ML Workspace
workspace = Workspace.create(name = workspace_name,
                             location = workspace_location,
                             resource_group = resource_group,
                             subscription_id = subscription_id,
                             exist_ok=True)

# Get the ACI Webservice containing our model
aks_webservice = None
for service in Webservice.list(workspace):
  if service.name == aks_webservice_name:
    aks_webservice = service

In [12]:
input_data = spark.sql("select * from bank_marketing_skewed order by rand()")

In [13]:
for chunk in np.array_split(input_data.toPandas(), chunks):
  print('.', end='')
  predictions = aks_webservice.run(chunk.to_json(orient='split'))

print('')
print('Scoring Complete!')

..........................................................................................................................................................................................................................................................
Scoring Complete!